In [1]:
import pandas as pd
import re
# import emoji

In [2]:
# pat = re.compile('RT @?(.+):')
pat = re.compile('RT @?([A-z0-9_]+):')

In [3]:
url = re.compile('https?:.*')

In [4]:
# emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002600-\U000027BF"
#                            "]", flags=re.UNICODE)
emoji_pattern = re.compile(r'\\[u|U][a-fA-F0-9]+[\\u|U[a-zA-F0-9]+]*')

In [13]:
df  = pd.read_csv('WorldCupDraw.csv',encoding='utf-8')
df.shape

(106548, 8)

In [6]:
df.head() 

,id,text,screen_name,retweet_count,favorite_count,friends_count,followers_count,url
0,937247505208827904,RT @BBCMOTD: Eden Hazard says the England and ...,SPonTheBall,35,0,81,25,https://t.co/0Ekz6haggd
1,937247428973088768,RT @stokecity: Legend! 👔😎 #WorldCupDraw #SCFC...,boothen_talers,396,0,294,287,https://t.co/NF4w2eDNXp
2,937246905980952576,RT @DFB_Team_EN: And South Korea round off Gro...,naneth_mandagi,1181,0,181,65,https://t.co/nCqErE1w9P
3,937246600803618816,RT @FCBayernUS: When @matshummels and @os_alan...,Drizzy__Fake,1582,0,109,306,https://t.co/udLBbxZPKM
4,937245866456428545,RT @kylewalker2: When you find out you gotta p...,egn_ho,3313,0,518,643,https://t.co/d1LQSMB5Nu


In [14]:
RT = []
Text = []
OTweeter = []
Emojis = []
for t in df['text']:
    retweet = t.split()[0]=='RT'
    ##print(t,retweet)
    RT.append(retweet)
    text = t
    if retweet:
        OTweeter.append(pat.match(t).group(1) if pat.match(t) else '')
        text = re.sub(pat,"",t)
    else:
        OTweeter.append('')
    text = ascii(text)
#     emojis_used = list()
#     t = ""
#     for c in text:
#         if c in emoji.UNICODE_EMOJI:
#             emojis_used.append(c)
#         else:
#             t = t + c
#     text = t
    emojis_used = emoji_pattern.findall(text)
    emojis = list()
#     print(t)
    for u in emojis_used:
        a = u.encode('utf8')
#         print(u,a)
        emojis.append(a.decode('unicode-escape'))
    
    emojis_used = "".join(emojis)
    text = re.sub(emoji_pattern,"",text)
    text = re.sub(url,"",text)
    
    text = text.replace('#WorldCupDraw','').strip()
    text = text.strip('\'')
    text = text.strip('\"\"\"')
#     text = text.encode('utf8')
    Text.append(text.strip())
    Emojis.append(emojis_used)

df['RT'] = RT
df['text'] = Text
df['original_tweeter'] = OTweeter
df['emojis_used'] = Emojis

In [15]:
df.head()

,id,text,screen_name,retweet_count,favorite_count,friends_count,followers_count,url,RT,original_tweeter,emojis_used
0,937247505208827904,Eden Hazard says the England and Belgium playe...,SPonTheBall,35,0,81,25,https://t.co/0Ekz6haggd,True,BBCMOTD,
1,937247428973088768,Legend! #SCFC,boothen_talers,396,0,294,287,https://t.co/NF4w2eDNXp,True,stokecity,👔😎🔴⚪️
2,937246905980952576,And South Korea round off Group F! What do yo...,naneth_mandagi,1181,0,181,65,https://t.co/nCqErE1w9P,True,DFB_Team_EN,🇰🇷
3,937246600803618816,When @matshummels and @os_alanis play against ...,Drizzy__Fake,1582,0,109,306,https://t.co/udLBbxZPKM,True,FCBayernUS,
4,937245866456428545,When you find out you gotta play @DeBruyneKev ...,egn_ho,3313,0,518,643,https://t.co/d1LQSMB5Nu,True,kylewalker2,


In [16]:
df.dtypes

id                   int64
text                object
screen_name         object
retweet_count        int64
favorite_count       int64
friends_count        int64
followers_count      int64
url                 object
RT                    bool
original_tweeter    object
emojis_used         object
dtype: object

In [17]:
df = df[df['text'].str.strip() != '']
df.shape

(106548, 11)

In [18]:
df.to_csv(path_or_buf='WorldCupDraw_Cleansed.csv',index=False,encoding='utf-8')

In [21]:
print(df.retweet_count.mean(),df.favorite_count.mean(),df.friends_count.mean(),df.followers_count.mean())

999.065134963 4.4556444044 1286.7842287 31680.6121466


In [22]:
df.id.nunique()

106548